<a href="https://colab.research.google.com/github/rjanow/Masterarbeit/blob/main/Masterarbeit_EDA_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aktuelle Version der EDA:
Die Struktur ist an die Gliederung des 2ten Materprojektes angepasst.

In [1]:
# Verbinden mit der Google-Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# import modules

import io, os, sys, setuptools, tokenize
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

from datetime import datetime
from datetime import timedelta
from math import sqrt

In [3]:
# Größe der Plots festlegen
matplotlib.rcParams['figure.figsize'] = (16, 9)

In [4]:
# Allgemeine Angaben
latitude = 50.2
longitude = 7.8

seconds_in_day = 24*60*60
seconds_in_year = (365.2425)*seconds_in_day

# tz, altitude, name = 'Europe/Berlin', 70, 'SanktAugustin'
# tus = pvlib.location.Location(latitude, longitude, tz, altitude, name)

In [5]:
# Pfad zur CSV-Datei auf Google Drive
name_Messwerte = 'Messdaten_CAMS_GHI.csv'
name_Vorhersage = 'Vorhersagedaten_CAMS_VarIdx.csv'

folder_import = '/content/drive/My Drive/Colab_Notebooks/Clean_Data/'
folder_export = '/content/drive/My Drive/Colab_Notebooks/plot_daily_UVI/'

# Import der Vorhersagewerte:

In [6]:
df_Vorhersage = pd.read_csv(folder_import + name_Vorhersage)
df_Vorhersage.set_index('Datetime', inplace=True)
df_Vorhersage.index = pd.to_datetime(df_Vorhersage.index)

# Import der Messwerte:

In [7]:
df_Messwerte = pd.read_csv(folder_import + name_Messwerte)
df_Messwerte.set_index('Datetime', inplace=True)
df_Messwerte.index = pd.to_datetime(df_Messwerte.index)

In [8]:
df_Messwerte['Observation_period'] = pd.to_datetime(df_Messwerte['Observation_period'])
df_Messwerte['date'] = df_Messwerte['Observation_period'].dt.date

In [9]:
def remove_sza_between_80_and_90(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    result = pd.DataFrame()

    for datum, df_tag in df.groupby('date'):
        # Zeilen entfernen, in denen SZA zwischen 80° und 100° liegt (exklusiv)
        df_filtered = df_tag[~df_tag['SZA'].between(70, 100)]
        result = pd.concat([result, df_filtered], ignore_index=True)

    return result

In [10]:
df_Messwerte = remove_sza_between_80_and_90(df_Messwerte)

NameError: name 'remove_sza_between_80_and_100' is not defined

In [ ]:
hour_bins = [3, 6, 9, 12, 15, 18, 21]  # Beispiel für mögliche Bins
hour_labels = ['03:00-06:00', '06:00-09:00', '09:00-12:00', '12:00-15:00', '15:00-18:00', '18:00-21:00']
df_Messwerte['Time_bins'] = pd.cut(df_Messwerte['Hour'], bins=hour_bins, labels=hour_labels, right=False)

In [ ]:
df_Messwerte

In [ ]:
df_Vorhersage

Allgemeine Informationen über die Vorhersagewerte und Messwerte:

In [ ]:
print(df_Vorhersage.describe(),
df_Vorhersage.isna().sum())

In [ ]:
df_Messwerte.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])

In [ ]:
# Interquartile Range berechnen

Q1_UVI = df_Messwerte['UVI'].quantile(0.25)
Q2_UVI = df_Messwerte['UVI'].quantile(0.5)
Q3_UVI = df_Messwerte['UVI'].quantile(0.75)

IQR_UVI = Q3_UVI - Q1_UVI

print(f"25. Perzentil: {round(Q1_UVI, 3)}, 5. Perzentil: {round(Q2_UVI, 3)}, 75. Perzentil: {round(Q3_UVI, 3)}, IQR: {round(IQR_UVI, 3)}")


In [ ]:
# Z-score über alle Messdaten berechnen
df_ZScore = pd.DataFrame()

mittelwert = df_Messwerte['UVI'].mean()
standardabweichung = df_Messwerte['UVI'].std()

df_ZScore['Z-Score'] = (df_Messwerte['UVI'] - mittelwert) / standardabweichung
plt.plot(df_ZScore)
plt.title('Z-Score des UVI')
plt.savefig('/content/drive/My Drive/Colab_Notebooks/plot_TSA/Z_Score_UVI.png')

# Qunatilspezifische Analyse des UVI:

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Datum in datetime, falls nicht bereits erfolgt
df_Messwerte['Datum'] = pd.to_datetime(df_Messwerte['Datum'])

# Monat extrahieren
df_Messwerte['Monat'] = df_Messwerte['Datum'].dt.month

# Uhrzeit binär in Kategorien gruppiert (falls noch nicht vorhanden)
# z. B. 06:00–09:00 etc. scheint schon in 'Time_bins' zu sein

In [ ]:
df_Messwerte, df_Vorhersage

In [ ]:
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_Messwerte, x='Time_bins', y='UVI')
plt.title('UVI nach Tageszeit (Time_bins)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.violinplot(data=df_Messwerte, x='Time_bins', y='UVI', inner='quartile')
plt.title('UVI Verteilung nach Tageszeit (Violinplot)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Stelle sicher, dass 'Monat' numerisch ist
df_Messwerte['Monat'] = df_Messwerte['Monat'].astype(int)

# Berechnung der Quantile und Median pro Monat
uvi_quantiles = df_Messwerte.groupby('Monat')['UVI'].quantile([0.1, 0.5, 0.9]).unstack()

plt.figure(figsize=(12, 6))

# Boxplot
sns.boxplot(data=df_Messwerte, x='Monat', y='UVI')

# Liniendiagramme für Quantile
plt.plot(uvi_quantiles.index - 1, uvi_quantiles[0.1], label='10%-Quantil', color='blue', linestyle='dashed', marker='o')
plt.plot(uvi_quantiles.index - 1, uvi_quantiles[0.5], label='Median', color='orange', linestyle='solid', marker='o')
plt.plot(uvi_quantiles.index - 1, uvi_quantiles[0.9], label='90%-Quantil', color='red', linestyle='solid', marker='o')

# Plot-Details
plt.title('UVI nach Monat mit 10%, 50% (Median) und 90%-Quantil')
plt.xlabel('Monat')
plt.ylabel('UVI')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.violinplot(data=df_Messwerte, x='Monat', y='UVI', inner='quartile')
plt.title('UVI Verteilung nach Monat (Violinplot)')
plt.tight_layout()
plt.show()

In [ ]:
df_Messwerte['Cloud_Index'] = df_Messwerte['ghi'] / df_Messwerte['Clear_sky_GHI']
# z. B. in Klassen einteilen
df_Messwerte['Bewölkung'] = pd.cut(df_Messwerte['Cloud_Index'],
                                   bins=[0, 0.2, 0.5, 0.8, 1.0, 1.5],
                                   labels=['sehr bewölkt', 'bewölkt', 'teilweise', 'klar', 'überklar'])

In [ ]:
sns.boxplot(data=df_Messwerte, x='Bewölkung', y='UVI')
plt.title('UVI nach geschätzter Bewölkung (Cloud Index)')
plt.show()

In [ ]:
quantiles = df_Messwerte.groupby('Time_bins')['UVI'].quantile([0.1, 0.5, 0.9]).unstack()
quantiles.columns = ['10%', '50%', '90%']
print(quantiles)

In [ ]:
# Heatmap 1: 90%-Quantil
pivot_90 = df_Messwerte.pivot_table(
    index='Time_bins',
    columns='Monat',
    values='UVI',
    aggfunc=lambda x: x.quantile(0.9)
)

# Heatmap 2: Median (50%-Quantil)
pivot_50 = df_Messwerte.pivot_table(
    index='Time_bins',
    columns='Monat',
    values='UVI',
    aggfunc='median'  # oder: lambda x: x.quantile(0.5)
)

# Plot: zwei Heatmaps nebeneinander
fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True)

# Heatmap für 90%-Quantil
sns.heatmap(pivot_90, annot=True, fmt=".2f", cmap="YlOrRd", ax=axes[0])
axes[0].set_title("90%-Quantil des UVI")
axes[0].set_ylabel("Tageszeit")
axes[0].set_xlabel("Monat")

# Heatmap für Median
sns.heatmap(pivot_50, annot=True, fmt=".2f", cmap="YlGnBu", ax=axes[1])
axes[1].set_title("Median des UVI")
axes[1.].set_ylabel("Tageszeit")
axes[1].set_xlabel("Monat")

plt.tight_layout()
plt.show()